In [ ]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import category_encoders as ce 
import numpy as np
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns

In [ ]:
df_test = pd.read_csv('https://raw.githubusercontent.com/DrIanGregory/Kaggle-GiveMeSomeCredit/master/data/GiveMeSomeCredit-testing.csv')
df_train = pd.read_csv('https://raw.githubusercontent.com/DrIanGregory/Kaggle-GiveMeSomeCredit/master/data/GiveMeSomeCredit-training.csv')
print(df_test.shape,
df_train.shape)

(101503, 12) (150000, 12)


In [ ]:
df_test.isna().sum()

Unnamed: 0                                   0
SeriousDlqin2yrs                        101503
RevolvingUtilizationOfUnsecuredLines         0
age                                          0
NumberOfTime30-59DaysPastDueNotWorse         0
DebtRatio                                    0
MonthlyIncome                            20103
NumberOfOpenCreditLinesAndLoans              0
NumberOfTimes90DaysLate                      0
NumberRealEstateLoansOrLines                 0
NumberOfTime60-89DaysPastDueNotWorse         0
NumberOfDependents                        2626
dtype: int64

In [ ]:
df_train.isna().sum()

Unnamed: 0                                  0
SeriousDlqin2yrs                            0
RevolvingUtilizationOfUnsecuredLines        0
age                                         0
NumberOfTime30-59DaysPastDueNotWorse        0
DebtRatio                                   0
MonthlyIncome                           29731
NumberOfOpenCreditLinesAndLoans             0
NumberOfTimes90DaysLate                     0
NumberRealEstateLoansOrLines                0
NumberOfTime60-89DaysPastDueNotWorse        0
NumberOfDependents                       3924
dtype: int64

In [ ]:
df_train['SeriousDlqin2yrs'].value_counts()


0    139974
1     10026
Name: SeriousDlqin2yrs, dtype: int64

In [ ]:
df = df_train.append(df_test, ignore_index = True)
df

<ipython-input-7-f58c5b142ca6>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_train.append(df_test, ignore_index = True)


,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1.0,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0.0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0.0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0.0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0.0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
251498,101499,NaN,0.282653,24,0,0.068522,1400.0,5,0,0,0,0.0
251499,101500,NaN,0.922156,36,3,0.934217,7615.0,8,0,2,0,4.0
251500,101501,NaN,0.081596,70,0,836.000000,NaN,3,0,0,0,NaN
251501,101502,NaN,0.335457,56,0,3568.000000,NaN,8,0,2,1,3.0


In [ ]:
df.isna().sum()

Unnamed: 0                                   0
SeriousDlqin2yrs                        101503
RevolvingUtilizationOfUnsecuredLines         0
age                                          0
NumberOfTime30-59DaysPastDueNotWorse         0
DebtRatio                                    0
MonthlyIncome                            49834
NumberOfOpenCreditLinesAndLoans              0
NumberOfTimes90DaysLate                      0
NumberRealEstateLoansOrLines                 0
NumberOfTime60-89DaysPastDueNotWorse         0
NumberOfDependents                        6550
dtype: int64

In [ ]:
df['SeriousDlqin2yrs'].value_counts()

0.0    139974
1.0     10026
Name: SeriousDlqin2yrs, dtype: int64

In [ ]:
df.loc[df['SeriousDlqin2yrs'].isna(), 'SeriousDlqin2yrs'] = 0
df['SeriousDlqin2yrs'].value_counts()

0.0    241477
1.0     10026
Name: SeriousDlqin2yrs, dtype: int64

In [ ]:
thrd_q = df['MonthlyIncome'].quantile(q=0.75)
fst_q = df['MonthlyIncome'].quantile(q=0.25)
df = df.loc[~df['MonthlyIncome'].isin(df[df['MonthlyIncome']>thrd_q+(thrd_q+fst_q)*1.5]['MonthlyIncome'])]

df['MonthlyIncome']= df['MonthlyIncome'].fillna(df['MonthlyIncome'].mean())
df['MonthlyIncome'].isna().sum()

<ipython-input-11-c3d8b9fb08b9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MonthlyIncome']= df['MonthlyIncome'].fillna(df['MonthlyIncome'].mean())


0

In [ ]:
df = df.drop(columns=['Unnamed: 0'])
df

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1.0,0.766127,45,2,0.802982,9120.00000,13,0,6,0,2.0
1,0.0,0.957151,40,0,0.121876,2600.00000,4,0,0,0,1.0
2,0.0,0.658180,38,1,0.085113,3042.00000,2,1,0,0,0.0
3,0.0,0.233810,30,0,0.036050,3300.00000,5,0,0,0,0.0
5,0.0,0.213179,74,0,0.375607,3500.00000,3,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
251498,0.0,0.282653,24,0,0.068522,1400.00000,5,0,0,0,0.0
251499,0.0,0.922156,36,3,0.934217,7615.00000,8,0,2,0,4.0
251500,0.0,0.081596,70,0,836.000000,6165.72365,3,0,0,0,NaN
251501,0.0,0.335457,56,0,3568.000000,6165.72365,8,0,2,1,3.0


In [ ]:
df.isna().sum()

SeriousDlqin2yrs                           0
RevolvingUtilizationOfUnsecuredLines       0
age                                        0
NumberOfTime30-59DaysPastDueNotWorse       0
DebtRatio                                  0
MonthlyIncome                              0
NumberOfOpenCreditLinesAndLoans            0
NumberOfTimes90DaysLate                    0
NumberRealEstateLoansOrLines               0
NumberOfTime60-89DaysPastDueNotWorse       0
NumberOfDependents                      6550
dtype: int64

In [ ]:
len(pd.unique(df['NumberOfDependents']))

15

In [ ]:
df['NumberOfDependents'].value_counts()

0.0     144753
1.0      43522
2.0      32426
3.0      15859
4.0       4763
5.0       1239
6.0        276
7.0        100
8.0         40
10.0         8
9.0          8
20.0         2
13.0         1
43.0         1
Name: NumberOfDependents, dtype: int64

In [ ]:
df= df.dropna()

In [ ]:

scaler = MinMaxScaler()
df['MonthlyIncome'] = scaler.fit_transform(df['MonthlyIncome'].values.reshape(-1,1))

<ipython-input-17-be8120b6e558>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MonthlyIncome'] = scaler.fit_transform(df['MonthlyIncome'].values.reshape(-1,1))


In [ ]:

df['NumberOfOpenCreditLinesAndLoans'] = scaler.fit_transform(df['NumberOfOpenCreditLinesAndLoans'].values.reshape(-1,1))
df['NumberRealEstateLoansOrLines'] = scaler.fit_transform(df['NumberRealEstateLoansOrLines'].values.reshape(-1,1))
df['DebtRatio'] = scaler.fit_transform(df['DebtRatio'].values.reshape(-1,1))
df['NumberOfDependents'] = scaler.fit_transform(df['NumberOfDependents'].values.reshape(-1,1))
df['age'] = scaler.fit_transform(df['age'].values.reshape(-1,1))

<ipython-input-18-45fda917bbbd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NumberOfOpenCreditLinesAndLoans'] = scaler.fit_transform(df['NumberOfOpenCreditLinesAndLoans'].values.reshape(-1,1))
<ipython-input-18-45fda917bbbd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NumberRealEstateLoansOrLines'] = scaler.fit_transform(df['NumberRealEstateLoansOrLines'].values.reshape(-1,1))
<ipython-input-18-45fda917bbbd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [ ]:
df.isna().sum()

SeriousDlqin2yrs                        0
RevolvingUtilizationOfUnsecuredLines    0
age                                     0
NumberOfTime30-59DaysPastDueNotWorse    0
DebtRatio                               0
MonthlyIncome                           0
NumberOfOpenCreditLinesAndLoans         0
NumberOfTimes90DaysLate                 0
NumberRealEstateLoansOrLines            0
NumberOfTime60-89DaysPastDueNotWorse    0
NumberOfDependents                      0
dtype: int64

In [ ]:
training_data = df.sample(frac=0.8, random_state=25)
testing_data = df.drop(training_data.index)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
#      x, y, test_size=0.2, random_state=42)

training_data = df.sample(frac=0.8, random_state=25)
testing_data = df.drop(training_data.index)


x_train, y_train = training_data.drop('SeriousDlqin2yrs',axis=1), training_data['SeriousDlqin2yrs']
x_test, y_test = testing_data.drop('SeriousDlqin2yrs',axis=1), testing_data['SeriousDlqin2yrs']

y_train=np.vstack(y_train.values).astype(np.float64)
y_test=np.vstack(y_test.values).astype(np.float64)


x_train, y_train, x_test, y_test = torch.tensor(x_train.values, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32), torch.tensor(x_test.values, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

In [ ]:
batch_size = 100

train_dl = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size)

test_dl = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [ ]:
x_train.shape

torch.Size([194398, 10])

In [ ]:
class CreditScoring(nn.Module):
    def __init__(self):
        super(CreditScoring, self).__init__()
        self.fc1 = nn.Linear(10, 64) # 10 - количество признаков
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

model = CreditScoring()

In [ ]:
%%time
# Определение функции потерь и оптимизатора
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Обучение модели
for epoch in range(3):
    for i, (inputs, labels) in enumerate(train_dl):
        optimizer.zero_grad()
        outputs = model(inputs)
        #print(outputs)
        loss = criterion(outputs.squeeze(), labels.squeeze())
        loss.backward()
        optimizer.step()

    # Тестирование модели
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_dl:
            outputs = model(inputs)
            predicted = torch.round(outputs)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print('Epoch {}, Accuracy: {}%'.format(epoch+1, accuracy))

Epoch 1, Accuracy: 95.87037037037037%
Epoch 2, Accuracy: 95.87448559670781%
Epoch 3, Accuracy: 95.8724279835391%
CPU times: user 17.8 s, sys: 67.3 ms, total: 17.9 s
Wall time: 22.4 s
